In [ ]:
from data.adult import Adult

adult = Adult()
train_X, train_y = adult.get_train_data()
test_X, test_y = adult.get_test_data()

task = adult.applicable_task()
task.train(train_X, train_y)
task.test(test_X, test_y)

In [ ]:
import numpy as np
from typing import List

alpha = 1.0
r = 1.7
gamma = 0.04
nu = 0.01
lambda_max = 20.0

thr_candidates: List[float] = np.linspace(0.0, 1.0, 200).tolist()

In [ ]:
from ferm_ge.algorithm_ge import ge_confmat

I_alpha_cache = []
err_cache = []

for thr in thr_candidates:
    _, confmat = task.predict_train_with_threshold(thr)
    tn, fp, fn, tp = confmat.astype(float)
    err: float = (fp + fn) / (tn + fp + fn + tp)
    err_cache.append(err)
    I_alpha: float = ge_confmat(alpha, r, tn, fp, fn, tp)
    I_alpha_cache.append(I_alpha)

In [ ]:
from ferm_ge.algorithm_gefair import GEFairSolverC

lib_path = GEFairSolverC.compile_gefair()
solver = GEFairSolverC(lib_path)

In [ ]:
gefair_result = solver.solve_gefair(
    thr_candidates,
    I_alpha_cache,
    err_cache,
    alpha,
    lambda_max,
    nu,
    r,
    gamma,
    collect_ge_history=True,
)

In [ ]:
from ferm_ge.utils import paramdict_to_frozenkey
from ferm_ge.metrics import calc_metrics

key = paramdict_to_frozenkey(
    {
        "alpha": alpha,
        "r": r,
        "gamma": gamma,
        "nu": nu,
        "lambda_max": lambda_max,
    }
)

gefair_metrics = calc_metrics(
    {key: gefair_result},
    task,
    repeat=0,
)

print("I_alpha", gefair_metrics[key].I_alpha)
print("err", gefair_metrics[key].err)

In [ ]:
from ferm_ge.plotting import plot_convergence

plot_convergence({key: gefair_result}, "I_alpha");
plot_convergence({key: gefair_result}, "err");